In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import combinations
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn import linear_model

import gc
import random
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from copy import deepcopy
from functools import partial

from scipy.optimize import minimize

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
filepath = '../data/'
df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col='Id')
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col="Id")
greeks = pd.read_csv(os.path.join(filepath, 'greeks.csv'), index_col="Id")

In [6]:
reg = linear_model.Ridge(alpha=.5)


In [12]:
df_train['EJ'] = df_train['EJ'].replace({'A': 0, 'B': 1})
df_test['EJ']  = df_test['EJ'].replace({'A': 0, 'B': 1})
#data = pd.concat([df_train, greeks], axis=1)
df_train.fillna(df_train.mean(), inplace=True)
target_col = 'Class'

X_train = df_train.drop([target_col],axis=1).reset_index(drop=True)
Y_train = df_train[target_col].reset_index(drop=True)
X_test = df_test.reset_index(drop=True)

#drop_cols = ['BC', 'CL']
#X_train.drop(drop_cols, axis=1, inplace=True)
#X_test.drop(drop_cols, axis=1, inplace=True)

# Only 'EJ' is a categorical data so far
#numeric_columns = [_ for _ in X_train.columns if _ not in ['EJ']]
#scaler = StandardScaler() # MinMaxScaler or StandardScaler
#X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
#X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

print(f"X_train shape :{X_train.shape} , y_train shape :{Y_train.shape}")
print(f"X_test shape :{X_test.shape}")

# Delete the train and test dataframes to free up memory
# del df_train, df_test

X_train.head(5)

class Splitter:
    def __init__(self, n_splits = 5):
        self.n_splits = n_splits
        
    def split_data(self, X, y, random_state_list):
        for random_state in random_state_list:
            # shuffle = True ???
            kfold = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
            for tr_index, val_index in kfold.split(X, y):
                yield tr_index, val_index

n_splits = 5
n_reapts = 1
random_state = 42
n_estimators = 99999
early_stopping_rounds = 1000
verbose = False
device = 'cpu'

# Fix seed
random.seed(random_state)
random_state_list = random.sample(range(9999), n_reapts)
for i, (tr_idx, val_idx) in enumerate(splitter.split_data(X_train, Y_train, random_state_list=random_state_list)):
    x_train, x_val = X_train.loc[tr_idx], X_train.loc[val_idx]
    y_train, y_val = Y_train.loc[tr_idx], Y_train.loc[val_idx]

X_train shape :(617, 56) , y_train shape :(617,)
X_test shape :(5, 56)


In [17]:
reg.fit(x_train, y_train)


Ridge(alpha=0.5)